[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fw-ai/cookbook/blob/main/learn/batch-api/batch_api.ipynb)

# Get Started with Batch API

## Introduction

The Batch API is designed for asynchronous processing of long-running API tasks by offloading execution and storing results for later retrieval. This architecture is ideal for workloads such as media processing, transcription, translation, and other time-intensive operations that benefit from deferred, non-blocking execution.

Clients submit requests synchronously over HTTP, specifying a target endpoint_id and path that define the backend service and API route. The server then processes these requests asynchronously in the background. Clients can check request status at any time and retrieve results once processing is complete. The system is fully endpoint-agnostic, allowing it to seamlessly route and support a wide range of backend services across Fireworks.AI's infrastructure.

For more information on Batch API parameters, including endpoint_id, path, and others, please refer to the link below:

- [Create Batch Request – Fireworks Docs](https://docs.fireworks.ai/api-reference/create-batch-request)  
- [Check Batch Status – Fireworks Docs](https://docs.fireworks.ai/api-reference/get-batch-status)

In this notebook, you will learn how to:

*  Submit a batch processing request to a backend service;
*  Check the status of your request and retrieve the results once processing is complete

## Install dependencies

In [ ]:
!pip3 install requests

## 1. Prepare audio Sample

In this example, we'll download a pre-recorded audio file and use it to test the Batch Inference API with a transcription request.

In [ ]:
!curl -L -o "audio.flac" "https://tinyurl.com/4997djsh"

## 2. Set up API credentials

To use the Fireworks Batch API, you'll need your API key. For security reasons, we'll get it from environment variables.

You can set your API key in the notebook by running:

In [ ]:
import os
os.environ["FIREWORKS_API_KEY"] = "your-api-key-here"

## 3. Submitting a Batch Processing Request

This section demonstrates how to submit a new request to the Batch API for asynchronous processing.

When constructing your request, you’ll need to specify the following key parameters:

* **`endpoint_id`**: Identifies the target backend service or model to handle the request (e.g., `"audio-prod"`, `"audio-turbo"`). This must be compatible with the selected operation or model type. You can refer to the [official documentation](https://docs.fireworks.ai/api-reference/create-batch-request) for a complete list of supported `endpoint_id`s and their corresponding services.

* **`path`**: The relative route of the target API operation (e.g., `"v1/audio/transcriptions"`, `"v1/audio/translations"`). This should correspond to a valid route supported by the backend service.

* **`payload`**: Contains the input data and configuration specific to the selected API route. Its structure should match the schema expected by the corresponding synchronous API.




The following example demonstrates how to send a batch processing request for the transcription service.

In [ ]:
import os
import requests

# === [Required by User] Define your input ===
audio_file_path = "audio.flac"
resource_path = "v1/audio/transcriptions"
endpoint_id = "audio-prod"
payload = {
    "model": "whisper-v3",
    "response_format": "verbose_json",
    "timestamp_granularities": "word",
    "diarize": "true"
}

# === [Environment and system settings] ===
api_key = os.environ.get("FIREWORKS_API_KEY")
batch_url = "https://batch.us-nevada-1.direct.fireworks.ai/"
url = batch_url + resource_path
params = {"endpoint_id": endpoint_id}

# === [Execution logic] Submit request and handle response ===

try:
    if not os.path.exists(audio_file_path):
        print(f"Error: Audio file '{audio_file_path}' not found.")
    else:
        files = {"file": open(audio_file_path, "rb")}
        headers = {"Authorization": api_key}

        response = requests.post(
            url,
            files=files,
            data=payload,
            headers=headers,
            params=params
        )

        if response.status_code == 200:
            result = response.json()
            batch_id = result.get("batch_id")
            print(f"Batch request successful. Batch ID: {batch_id}")
        else:
            print(f"Error: {response.status_code} - {response.text}")

except Exception as e:
    print(f"Unexpected error: {str(e)}")



## 4. Check Batch Request Status or Retrieve Results

To check the status or retrieve the results of a batch request, you’ll need to query the Batch API using your unique identifiers. Each request must include:

- **`account_id`**: This can be found on your [Fireworks AI account homepage](https://fireworks.ai/account/home)

- **`batch_id`**: This was returned in the response when you initially submitted the batch request

In [ ]:
import os
import requests
import json

# === [Required by User] Define your input ===
account_id = "<your_account_id>"
batch_id = "<your_batch_id>"

# === [System constants] ===
api_key = os.environ.get("FIREWORKS_API_KEY")
url = f"https://batch.us-nevada-1.direct.fireworks.ai/v1/accounts/{account_id}/batch_job/{batch_id}"
headers = {"Authorization": api_key}

# === [Request] Fetch batch status or result ===
try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    result = response.json()
    print("Batch Status:", result.get("status"))
    print("Full Response:")
    print(json.dumps(result, indent=2))

except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")

## Conclusion

In this notebook, you learned how to use the Fireworks.AI Batch API to process long-running requests asynchronously.
We covered how to submit batch jobs, check their status, and retrieve results once processing is complete.

This approach is especially useful for tasks such as transcription, translation, and other media-related workloads that benefit from deferred execution.

For more information, see:

- [Create Batch Request – Fireworks Docs](https://docs.fireworks.ai/api-reference/create-batch-request)  
- [Check Batch Status – Fireworks Docs](https://docs.fireworks.ai/api-reference/get-batch-status)

Explore the community or reach out to us in [discord](https://discord.gg/fireworks-ai).